# 📊 Análise de Batalhas - Clash Royale + MongoDB

In [ ]:
from pymongo import MongoClient
from datetime import datetime
from config import MONGO_URI, DB_NAME

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]




## 🔍 Consulta 1 — % de Vitórias/Derrotas com uma Carta Específica

In [44]:
# Parâmetros da consulta
carta = "Rocket"
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Pipeline
pipeline1 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim},
            "$or": [
                {"team.cards.name": carta},
                {"opponent.cards.name": carta}
            ]
        }
    },
    {
        "$group": {
            "_id": "$is_winner",
            "total": {"$sum": 1}
        }
    }
]

# Execução
result1 = list(battles.aggregate(pipeline1))

# Processar resultados
vitorias = sum(doc["total"] for doc in result1 if doc["_id"] == True)
derrotas = sum(doc["total"] for doc in result1 if doc["_id"] == False)
total = vitorias + derrotas

print(f"Carta: {carta}")
print(f"Período: {inicio.date()} até {fim.date()}")

if total == 0:
    print("Nenhuma batalha encontrada com essa carta no intervalo fornecido.")
else:
    print(f"Total de batalhas: {total}")
    print(f"Vitórias: {vitorias} ({(vitorias/total)*100:.2f}%)")
    print(f"Derrotas: {derrotas} ({(derrotas/total)*100:.2f}%)")


Carta: Rocket
Período: 2025-04-01 até 2025-04-30
Total de batalhas: 32
Vitórias: 14 (43.75%)
Derrotas: 18 (56.25%)


## 🔍 Consulta 2 — Decks com mais de X% de vitórias

In [43]:
# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)
min_porcentagem_vitoria = 60  # Exemplo: 60%

# Conexão
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]

# Pipeline
pipeline2 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim}
        }
    },
    {
        "$project": {
            "deck": {
                "$sortArray": {
                    "input": {
                        "$map": {
                            "input": {
                                "$ifNull": [
                                    { "$getField": {
                                        "field": "cards",
                                        "input": { "$arrayElemAt": ["$team", 0] }
                                    }},
                                    []
                                ]
                            },
                            "as": "card",
                            "in": "$$card.name"
                        }
                    },
                    "sortBy": 1
                }
            },
            "is_winner": 1
        }
    },
    {
        "$group": {
            "_id": "$deck",
            "total": {"$sum": 1},
            "vitorias": {
                "$sum": {
                    "$cond": [{"$eq": ["$is_winner", True]}, 1, 0]
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "deck": "$_id",
            "total": 1,
            "vitorias": 1,
            "porcentagem": {
                "$multiply": [
                    {"$divide": ["$vitorias", "$total"]},
                    100
                ]
            }
        }
    },
    {
        "$match": {
            "porcentagem": {"$gt": min_porcentagem_vitoria}
        }
    },
    {
        "$sort": {"porcentagem": -1}
    }
]


# Execução  
result2 = list(battles.aggregate(pipeline2))

# Exibição
print(f"Decks com mais de {min_porcentagem_vitoria}% de vitórias entre {inicio.date()} e {fim.date()}")
for deck in result2:
    print(f"Deck: {deck['deck']}")
    print(f"Vitórias: {deck['vitorias']} / {deck['total']} ({deck['porcentagem']:.2f}%)\n")


Decks com mais de 60% de vitórias entre 2025-04-01 e 2025-04-30
Deck: ['Bats', 'Electro Giant', 'Lightning', 'Lumberjack', 'Minion Horde', 'Tesla', 'The Log', 'Valkyrie']
Vitórias: 2 / 2 (100.00%)



## 🔍 Consulta 3 — Derrotas com combo de cartas (X1, X2, ...) em um intervalo de tempo

In [42]:
# Parâmetros da consulta
combo = ["Knight", "Zap"]
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Conexão
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]

# Pipeline
pipeline3 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim},
            "is_winner": False  # derrota do player (você)
        }
    },
    {
        "$project": {
            "cartas": {
                "$map": {
                    "input": {
                        "$ifNull": [
                            { "$getField": {
                                "field": "cards",
                                "input": { "$arrayElemAt": ["$team", 0] }
                            }},
                            []
                        ]
                    },
                    "as": "card",
                    "in": "$$card.name"
                }
            }
        }
    },
    {
        "$match": {
            "cartas": {
                "$all": combo  # verificar se TODAS as cartas do combo estão presentes
            }
        }
    },
    {
        "$count": "total_derrotas"
    }
]

# Execução
result3 = list(battles.aggregate(pipeline3))

# Exibição
print(f"Combo: {combo}")
print(f"Período: {inicio.date()} até {fim.date()}")
if result3:
    print(f"Total de derrotas com esse combo: {result3[0]['total_derrotas']}")
else:
    print("Nenhuma derrota encontrada com esse combo no intervalo.")

Combo: ['Knight', 'Zap']
Período: 2025-04-01 até 2025-04-30
Total de derrotas com esse combo: 19


## 🔍 Consulta 4 — Vitórias com a carta X, onde o player venceu com menos troféus que o adversário:

In [ ]:
# Parâmetros
carta = "Knight"
inicio = datetime(2025, 3, 1)
fim = datetime(2025, 4, 30)

pipeline4 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim},
            "is_winner": True
        }
    },
    {
        "$project": {
            "teamTrophies": { "$arrayElemAt": ["$team.startingTrophies", 0] },
            "opponentTrophies": { "$arrayElemAt": ["$opponent.startingTrophies", 0] },
            "opponentCrowns": { "$arrayElemAt": ["$opponent.crowns", 0] },
            "tem_carta": {
                "$in": [carta, {
                    "$map": {
                        "input": {
                            "$ifNull": [{ "$arrayElemAt": ["$team.cards", 0] }, []]
                        },
                        "as": "card",
                        "in": "$$card.name"
                    }
                }]
            }
        }
    },
    {
        "$match": {
            "$expr": {
                "$and": [
                    { "$lt": ["$teamTrophies", "$opponentTrophies"] },
                    { "$gte": ["$opponentCrowns", 2] },
                    { "$eq": ["$tem_carta", True] }
                ]
            }
        }
    },
    {
        "$count": "total"
    }
]



result4 = list(battles.aggregate(pipeline4))

print(f"Carta: {carta}")
print(f"Período: {inicio.date()} até {fim.date()}")
if result4:
    print(f"Total de vitórias com esses critérios: {result4[0]['total']}")
else:
    print("Nenhuma vitória encontrada com esses critérios.")

Carta: Knight
Período: 2025-03-01 até 2025-04-30
Total de vitórias com esses critérios: 4


### 🔍 Consulta 5 — Combos de N cartas (ex: 3 cartas) que tiveram mais de Y% de vitórias dentro de um intervalo de tempo.


In [ ]:
from pymongo import MongoClient
from datetime import datetime
from config import MONGO_URI, DB_NAME
from itertools import combinations
from collections import defaultdict

# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)
combo_size = 3  # N = tamanho do combo
min_porcentagem_vitoria = 70  # Y = % mínima de vitórias

# Conexão
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
battles = db["battles"]

# Buscar todas as batalhas no período
matches = list(battles.find({
    "utcTime": {"$gte": inicio, "$lte": fim}
}))

# Contadores de combos
combo_stats = defaultdict(lambda: {"total": 0, "vitorias": 0})

# Processar cada batalha
for match in matches:
    team = match.get("team", [])
    if not team or "cards" not in team[0]:
        continue

    cards = sorted(card["name"] for card in team[0]["cards"])
    if len(cards) != 8:
        continue

    is_winner = match.get("is_winner", False)

    for combo in combinations(cards, combo_size):
        combo_stats[combo]["total"] += 1
        if is_winner:
            combo_stats[combo]["vitorias"] += 1

# Filtrar combos com % de vitórias > Y
result5 = []
for combo, stats in combo_stats.items():
    total = stats["total"]
    vitorias = stats["vitorias"]
    porcentagem = (vitorias / total) * 100
    if porcentagem > min_porcentagem_vitoria:
        result5.append({
            "combo": combo,
            "vitorias": vitorias,
            "total": total,
            "porcentagem": porcentagem
        })

# Ordenar por % de vitória (desc)
result5.sort(key=lambda x: x["porcentagem"], reverse=True)

# Exibir
print(f"Combos de {combo_size} cartas com mais de {min_porcentagem_vitoria}% de vitórias entre {inicio.date()} e {fim.date()}")
for combo in result5:
    print(f"Combo: {combo['combo']}")
    print(f"Vitórias: {combo['vitorias']} / {combo['total']} ({combo['porcentagem']:.2f}%)\n")


Combos de 3 cartas com mais de 70% de vitórias entre 2025-04-01 e 2025-04-30
Combo: ('Bats', 'Electro Giant', 'Lightning')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Electro Giant', 'Lumberjack')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Electro Giant', 'Minion Horde')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Electro Giant', 'Tesla')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Electro Giant', 'The Log')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Electro Giant', 'Valkyrie')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lightning', 'Lumberjack')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lightning', 'Minion Horde')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lightning', 'Tesla')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lightning', 'The Log')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lightning', 'Valkyrie')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lumberjack', 'Minion Horde')
Vitórias: 2 / 2 (100.00%)

Combo: ('Bats', 'Lumberjack', 'Tesla')
Vitórias: 2 / 2 (100.00%)

Combo: 

### 📉 Consulta Extra 1 — Decks Mais Utilizados no Período


In [63]:
# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Pipeline
pipeline_extra = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim}
        }
    },
    {
        "$project": {
            "deck": {
                "$sortArray": {
                    "input": {
                        "$map": {
                            "input": {
                                "$ifNull": [
                                    {
                                        "$getField": {
                                            "field": "cards",
                                            "input": { "$arrayElemAt": ["$team", 0] }
                                        }
                                    },
                                    []
                                ]
                            },
                            "as": "card",
                            "in": "$$card.name"
                        }
                    },
                    "sortBy": 1
                }
            }
        }
    },
    {
        "$match": {
            "$expr": {
                "$eq": [{ "$size": "$deck" }, 8]
            }
        }
    },
    {
        "$group": {
            "_id": "$deck",
            "quantidade": { "$sum": 1 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "deck": "$_id",
            "quantidade": 1
        }
    },
    {
        "$sort": { "quantidade": -1 }
    }
]


# Execução
result_extra = list(battles.aggregate(pipeline_extra))

# Exibição
print(f"Decks mais usados entre {inicio.date()} e {fim.date()}")
for deck in result_extra:
    print(f"Deck: {deck['deck']}")
    print(f"Usado: {deck['quantidade']} vezes\n")


Decks mais usados entre 2025-04-01 e 2025-04-30
Deck: ['Executioner', 'Guards', 'Inferno Tower', 'Knight', 'Rocket', 'Royal Giant', 'Tornado', 'Zap']
Usado: 32 vezes

Deck: ['Bats', 'Electro Giant', 'Lightning', 'Lumberjack', 'Minion Horde', 'Tesla', 'The Log', 'Valkyrie']
Usado: 2 vezes

Deck: ['Freeze', 'Giant Snowball', 'Goblins', 'Mega Knight', 'Miner', 'Mini P.E.K.K.A', 'Skeleton Barrel', 'Spear Goblins']
Usado: 2 vezes

Deck: ['Arrows', 'Baby Dragon', 'Bandit', 'Electro Wizard', 'Goblin Barrel', 'Minions', 'P.E.K.K.A', 'Skeleton Army']
Usado: 2 vezes



### 📉 Consulta Extra 2 — Cartas Mais Usadas em Derrotas

In [68]:
# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

# Pipeline
pipeline_extra2 = [
    {
        "$match": {
            "is_winner": False,
            "utcTime": {"$gte": inicio, "$lte": fim}
        }
    },
    {
        "$project": {
            "cartas": {
                "$map": {
                    "input": {
                        "$getField": {
                            "field": "cards",
                            "input": { "$arrayElemAt": ["$team", 0] }
                        }
                    },
                    "as": "card",
                    "in": "$$card.name"
                }
            }
        }
    },
    { "$unwind": "$cartas" },
    {
        "$group": {
            "_id": "$cartas",
            "derrotas": { "$sum": 1 }
        }
    },
    { "$sort": { "derrotas": -1 } },
    { "$limit": 10 }
]

# Execução
result_extra2 = list(battles.aggregate(pipeline_extra2))
print(f"Cartas mais comuns em derrotas entre {inicio.date()} e {fim.date()}")
for doc in result_extra2:
    print(f"{doc['_id']}: {doc['derrotas']} derrotas")


Cartas mais comuns em derrotas entre 2025-04-01 e 2025-04-30
Inferno Tower: 18 derrotas
Executioner: 18 derrotas
Knight: 18 derrotas
Zap: 18 derrotas
Tornado: 18 derrotas
Guards: 18 derrotas
Rocket: 17 derrotas
Royal Giant: 17 derrotas
Mega Knight: 2 derrotas
Freeze: 2 derrotas


### 📉 Consulta Extra 3 — Cartas com Maior Taxa de Vitória Individual


In [69]:
# Parâmetros
inicio = datetime(2025, 4, 1)
fim = datetime(2025, 4, 30)

pipeline_extra3 = [
    {
        "$match": {
            "utcTime": {"$gte": inicio, "$lte": fim}
        }
    },
    {
        "$project": {
            "is_winner": 1,
            "cartas": {
                "$map": {
                    "input": {
                        "$getField": {
                            "field": "cards",
                            "input": { "$arrayElemAt": ["$team", 0] }
                        }
                    },
                    "as": "card",
                    "in": "$$card.name"
                }
            }
        }
    },
    { "$unwind": "$cartas" },
    {
        "$group": {
            "_id": "$cartas",
            "total": { "$sum": 1 },
            "vitorias": {
                "$sum": { "$cond": ["$is_winner", 1, 0] }
            }
        }
    },
    {
        "$project": {
            "taxa_vitoria": {
                "$multiply": [
                    { "$divide": ["$vitorias", "$total"] },
                    100
                ]
            },
            "total": 1,
            "vitorias": 1
        }
    },
    { "$match": { "total": { "$gte": 5 } } },  # ignora cartas com poucos usos
    { "$sort": { "taxa_vitoria": -1 } },
    { "$limit": 10 }
]

# Execução
result_extra3 = list(battles.aggregate(pipeline_extra3))
print(f"Cartas com maior taxa de vitória entre {inicio.date()} e {fim.date()}")
for doc in result_extra3:
    print(f"{doc['_id']}: {doc['taxa_vitoria']:.2f}% de vitória (usada {doc['total']} vezes)")


Cartas com maior taxa de vitória entre 2025-04-01 e 2025-04-30
Tornado: 47.06% de vitória (usada 34 vezes)
Executioner: 47.06% de vitória (usada 34 vezes)
Guards: 47.06% de vitória (usada 34 vezes)
Zap: 47.06% de vitória (usada 34 vezes)
Inferno Tower: 47.06% de vitória (usada 34 vezes)
Knight: 47.06% de vitória (usada 34 vezes)
Royal Giant: 46.88% de vitória (usada 32 vezes)
Rocket: 46.88% de vitória (usada 32 vezes)
